### Importing Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats

### Loading Data

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
data = pd.concat([X, y], axis = 1)

In [ ]:
adult.variables

### EDA

In [ ]:
data.info()

#### Exploring income encoding

In [ ]:
# Class imbalance, preprocessing for income column needed
# Consider using f1 score instead of accuracy 
y.assign(count = y["income"]).groupby("income").count()

#### Exploring missing and unknown values

In [ ]:
# All people with missing workclass also has missing occupation
data[data["workclass"].isna()]["occupation"].unique()

In [ ]:
# People with missing occupation either had unknown occupation or never-worked
data[data["occupation"].isna()]["workclass"].unique()

In [ ]:
# What is the ? value
data["workclass"].unique()

In [ ]:
# People with unknown workclass, occupation, and native-country is marked with ?
data[data["workclass"] == "?"]

In [ ]:
# Which columns have ? values
for col in data.columns:
    if "?" in np.array(data[col]):
        print(col)

#### Exploring how many rows impacted if drop missing / ? values

In [ ]:
data[data["workclass"] == "?"]["occupation"].unique()

In [ ]:
data[data["workclass"] == "?"]["native-country"].unique()

In [ ]:
data[data["occupation"] == "?"]["workclass"].unique()

In [ ]:
data[data["occupation"] == "?"]["native-country"].unique()

In [ ]:
data[data["native-country"] == "?"]["workclass"].unique()

In [ ]:
data[data["native-country"] == "?"]["occupation"].unique()

In [ ]:
# Dropping doesn't impact much
data[(data["workclass"] != "?") | (data["occupation"] != "?") | (data["native-country"] != "?")].dropna()

#### Exploring possibly correlated features

In [ ]:
# education-num can be used to encode education
data.groupby(["education", "education-num"])[["age"]].count()

#### Exploring how many of each categorical variable there are and distribution

In [ ]:
# Number of classes in each categorical cariable
categorical = data.drop(columns = ["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", 
                                   "hours-per-week", "income"])
categorical_count = {}
for col in categorical.columns:
    categorical_count[col] = len(categorical[col].unique())
    
categorical_count

In [ ]:
# Function to get count of classes in each feature
def get_distr(col):
    return data.groupby(col)[["age"]].count().sort_values("age")

In [ ]:
get_distr("workclass")

In [ ]:
get_distr("education")

In [ ]:
get_distr("marital-status")

In [ ]:
get_distr("occupation")

In [ ]:
get_distr("relationship")

In [ ]:
get_distr("race")

In [ ]:
get_distr("sex")

In [ ]:
get_distr("native-country")

#### Exploring capital gain and capital loss

In [ ]:
data["capital-gain"].describe()

In [ ]:
data["capital-loss"].describe()

### Preprocessing

1. Replace nan with ? to represent unknown category or drop all missing and ? values
2. Preprocess income to be a binary value
3. Drop education column since education-num encodes that

In [ ]:
# Encodes if you made over 50k as 1 and under 50k as 0
data = data.assign(income = data["income"].str.split(".").str[0].apply(lambda x: 1 if x[0] == "<" else 0))